In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dataloader
from tqdm import tqdm

from model_architecture import Network

In [2]:
PROPOSAL_SIZE = (64, 64)

dataset_train = dataloader.PotholeDataset('../Potholes/annotated-images/', '../Potholes/proposals/', '../Potholes/annotated-images/', proposals_per_batch=5, proposal_size=PROPOSAL_SIZE, split='train')
dataset_val = dataloader.PotholeDataset('../Potholes/annotated-images/', '../Potholes/proposals/', '../Potholes/annotated-images/', proposals_per_batch=5, proposal_size=PROPOSAL_SIZE, split='val')
dataset_test = dataloader.PotholeDataset('../Potholes/annotated-images/', '../Potholes/proposals/', '../Potholes/annotated-images/', proposals_per_batch=5, proposal_size=PROPOSAL_SIZE, split='test')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)

model = Network()
model.apply(init_weights)
model.to(device)
#Initialize the optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.1)

cuda


In [4]:
def train(model, optimizer, num_epochs=10):
    def loss_fun(output, target):
        return F.cross_entropy(output, target)
    
    out_dict = {
              'train_acc': [],
              'val_acc': [],
              'train_loss': [],
              'val_loss': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        train_correct = 0
        train_loss = []
        # for minibatch_no, (data, target) in tqdm(enumerate(dataset), total=len(dataset)):
        for idx, (single_image_dict) in tqdm(enumerate(dataset_train), total=len(dataset_train)):
            # print(single_image_dict['labels'])
            # for proposal, label, proposal_image in zip(single_image_dict['proposals'], single_image_dict['labels'], single_image_dict['proposal_images']):
            # print(label)
            proposal_image, label = single_image_dict['proposal_images'].to(device), single_image_dict['labels'].to(device)
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            output = model(proposal_image)
            #Compute the loss

            # print(f"Proposal shape: {single_image_dict['proposals'].shape if single_image_dict['proposals'] is not None else 'None'}")
            # print(f"Proposal image shape: {proposal_image.shape if proposal_image is not None else 'None'}")
            # print(f"Label: {label}")
            # print(f"Label shape: {label.shape if isinstance(label, torch.Tensor) else 'Not a Tensor'}")

            loss = loss_fun(output, label)
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()

            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = output.argmax(1)
            train_correct += (label==predicted).sum().cpu().item()

        #Comput the test accuracy
        val_loss = []
        val_correct = 0
        model.eval()
        for single_val_dict in dataset_val:
            # for proposal_val, label_val, proposal_image_val in zip(single_val_dict['proposals'], single_val_dict['labels'], single_val_dict['proposal_images']):
            proposal_image_val, label_val = single_val_dict['proposal_images'].to(device), single_val_dict['labels'].to(device)
            with torch.no_grad():
                output = model(proposal_image_val)

            val_loss.append(loss_fun(output, label_val).cpu().item())
            predicted = output.argmax(1)
            val_correct += (label_val==predicted).sum().cpu().item()

        out_dict['train_acc'].append(train_correct/len(dataset_train))
        out_dict['val_acc'].append(val_correct/len(dataset_val))
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['val_loss'].append(np.mean(val_loss))

        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(val_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100/5:.1f}%\t test: {out_dict['val_acc'][-1]*100/5:.1f}%") # Dividing by 5 because of the batch_size
        
    return out_dict

In [5]:
train(model, optimizer)

  0%|          | 0/10 [00:00<?, ?epoch/s]c:\Users\samer\Documents\Intro to deep learning for computer vision\pothole_detector\dataloader.py:153: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels[selected_indices])
 10%|█         | 1/10 [00:09<01:26,  9.59s/epoch]

Loss train: 0.427	 test: 0.426	 Accuracy train: 90.3%	 test: 89.5%


 20%|██        | 2/10 [00:17<01:07,  8.45s/epoch]

Loss train: 0.423	 test: 0.440	 Accuracy train: 89.6%	 test: 87.7%


 30%|███       | 3/10 [00:24<00:56,  8.04s/epoch]

...

 90%|█████████ | 9/10 [01:11<00:07,  7.77s/epoch]

Loss train: 0.424	 test: 0.439	 Accuracy train: 89.3%	 test: 87.7%


100%|██████████| 10/10 [01:18<00:00,  7.88s/epoch]

Loss train: 0.404	 test: 0.452	 Accuracy train: 91.4%	 test: 86.4%


{'train_acc': [4.514563106796117,
  4.480582524271845,
  4.606796116504855,
  4.524271844660194,
  4.577669902912621,
  4.475728155339806,
  4.480582524271845,
  4.572815533980583,
  4.466019417475728,
  4.567961165048544],
 'val_acc': [4.4772727272727275,
  4.386363636363637,
  4.340909090909091,
  4.590909090909091,
  4.568181818181818,
  4.568181818181818,
  4.4772727272727275,
  4.659090909090909,
  4.386363636363637,
  4.318181818181818],
 'train_loss': [0.42712912354075794,
  0.423326759662443,
  0.3966262838215504,
  0.41252137706117725,
  0.4015305144983588,
  0.4218972374513311,
  0.4206112866262788,
  0.4031321796977404,
  0.4239137090236238,
  0.40367281726263105],
 'val_loss': [0.42559423026713455,
  0.43983467803760007,
  0.4482715278863907,
  0.3990793146870353,
  0.40325781364332547,
  0.4024789665233005,
  0.4217609390616417,
  0.38459455357356503,
  0.4387066052718596,
  0.4517950341105461]}